# Use OpenAPI with Managed Online Endpoints

This example demonstrates how to work with with OpenAPI and Managed Online Endpoints using both automatically-generated and custom Swagger files. 

The AzureML Inference Server automatically generates swagger files for scoring scripts that use [Inference Schema](https://github.com/Azure/InferenceSchema). In this example, a simple Inference Schema-decorated [scoring script](openapi/decorated/code/score.py) is used. For more complex examples, refer to the [Inference Schema example](../inference-schema).  

## 1. Configure parameters, assets, and clients

### 1.1 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

### 1.2 Set endpoint details

In [ ]:
import random

endpoint_name = f"endpt-moe-{random.randint(0,10000)}"

### 1.3 Create an MLClient instance

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
    BuildContext,
    ProbeSettings,
)
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## 2. Create an endpoint

### 2.1 Define and create the endpoint

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
poller = ml_client.online_endpoints.begin_create_or_update(endpoint)
poller.wait()

### 2.2 Confirm that creation was successful

In [ ]:
status = poller.status()
if status != "Succeeded":
    raise Exception(status)
else:
    print("Endpoint creation succeeded")
    endpoint = poller.result()
    print(endpoint)

### 2.3 Get endpoint key

In [ ]:
key = ml_client.online_endpoints.get_keys(endpoint_name).primary_key

## 3. Create Deployment: Auto-Generated Swagger

### 3.1 Define the deployment

In [ ]:
deployment = ManagedOnlineDeployment(
    name="openapi",
    endpoint_name=endpoint_name,
    model=Model(path="../model-1/model"),
    code_configuration=CodeConfiguration(
        code="openapi/code-decorated", scoring_script="score.py"
    ),
    environment=Environment(
        image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference",
        conda_file="openapi/env.yml",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

### 3.2 Create the deployment

In [ ]:
poller = ml_client.online_deployments.begin_create_or_update(deployment)
poller.wait()

### 3.3 Confirm that creation was successful

In [ ]:
status = poller.status()
if status != "Succeeded":
    raise Exception(status)
else:
    print("Deployment creation succeeded")
    deployment = poller.result()
    print(deployment)

### 3.4 Set traffic to 100% 

In [ ]:
endpoint.traffic = {"openapi": 100}
endpoint = ml_client.begin_create_or_update(endpoint).result()

### 3.5 Test endpoint

In [ ]:
ml_client.online_endpoints.invoke(endpoint_name, request_file="../model-1/sample-request.json")

### 3.6 Get Swagger (Default Version)

Swagger files are made available by default at the API endpoint `/swagger.json`. The specific route for an endpoint can be retrieved from the `openapi_uri` attribute of the endpoint. 

In [ ]:
import requests

res = requests.get(url=endpoint.openapi_uri, headers={"Authorization": f"Bearer {key}"})
print(res.content)

### 3.7 Get Swagger (Specify Version)
Specific versions can be retrieved by adding a `version` parameter to the request. 

In [ ]:
import requests

res = requests.get(
    url=endpoint.openapi_uri,
    params={"version": 3},
    headers={"Authorization": f"Bearer {key}"},
)
print(res.content)

## 4. Create Deployment: Custom Swagger

Custom swagger files can be integrated by including them at the root of the `code` directory. The custom file should be named `swagger<version>.json` i.e. `swagger2.json`. 

### 4.1 Define the deployment

In [ ]:
deployment = ManagedOnlineDeployment(
    name="openapi",
    endpoint_name=endpoint_name,
    model=Model(path="../model-1/model"),
    code_configuration=CodeConfiguration(
        code="openapi/code-custom", scoring_script="score.py"
    ),
    environment=Environment(
        image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference",
        conda_file="openapi/env.yml",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

### 4.2 Update the deployment

In [ ]:
poller = ml_client.online_deployments.begin_create_or_update(deployment)
poller.wait()

### 4.3 Confirm update was successful

In [ ]:
status = poller.status()
if status != "Succeeded":
    raise Exception(status)
else:
    print("Deployment creation succeeded")
    deployment = poller.result()
    print(deployment)

### 4.4 Test endpoint

In [ ]:
ml_client.online_endpoints.invoke(endpoint_name, request_file="../model-1/sample-request.json")

### 4.5 Get Custom Swagger

In [ ]:
import requests

res = requests.get(url=endpoint.openapi_uri, headers={"Authorization": f"Bearer {key}"})
print(res.content)

## 5. Delete assets

### 5.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)